In [2]:
import re
import string
from collections import Counter
import squarify
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
from spacy.tokenizer import Tokenizer

from bs4 import BeautifulSoup
import html as ihtml

import requests

In [4]:
def extract_job_title(soup):
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [5]:
def extract_company(soup):
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-linl-source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)

In [6]:
def extract_location(soup):
    locations = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            locations.append(div.find(name="span", attrs={"class":"location"}).text)
        except:
            locations.append("None")
    return(locations)

In [7]:
def extract_salary(soup):
    salaries = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            salaries.append(div.find(name="span", attrs={"class":"salaryText"}).text.replace("\n",""))
        except:
            salaries.append("None")
    return(salaries)

In [9]:
def extract_job_title(soup):
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [10]:
def extract_url(soup):
    urls = []
    for div1 in soup.find_all(name="div", attrs={"class":"row"}):
        for div2 in div1.find_all(name="div", attrs={"class":"title"}):
            for a in div2.find_all(name="a", href=True):
                urls.append(a['href'])
    return(urls)

def extract_desc(urls):
    descs = []
    for url in urls:
        full_url = "https://www.indeed.com" + url
        detail_page = requests.get(full_url)
        detail_soup = BeautifulSoup(detail_page.text, "html.parser")
        
        for div in detail_soup.find_all(name="div", attrs={"id":"jobDescriptionText"}):
            descs.append(div.text)
    return(descs)

In [11]:
def extract_count(soup):
    for div in soup.find_all(name="div", attrs={"id":"searchCountPages"}):
        temp_str = div.text.replace(" ", "")
        count_str = re.search('of(.*)jobs', temp_str) 
        count = int(count_str.group(1))
    return(count)

In [16]:
max_results = 100
title_name = "data+scientist"
city_name = "San+Francisco"
st_name = "CA"
columns = ["city", "job_title", "company", "location", "salary", "description"]

city_url = "https://www.indeed.com/jobs?q=" + title_name + \
               "&l=" + city_name + "%2C+" + st_name
page = requests.get(city_url)
soup = BeautifulSoup(page.text, "html.parser")
max_counter = extract_count(soup)
counter = 0
print(max_counter)

job_title_list = []
company_list = []
location_list = []
salary_list = []
desc_list = []

for start in range(0, max_results, 10):
    city_url = "https://www.indeed.com/jobs?q=" + title_name + \
               "&l=" + city_name + "%2C+" + st_name + \
               "&start=" + str(start)
    print(city_url)
    page = requests.get(city_url)
    soup = BeautifulSoup(page.text, "html.parser")

    job_title_list.extend(extract_job_title(soup))
    company_list.extend(extract_company(soup))
    location_list.extend(extract_location(soup))
    salary_list.extend(extract_salary(soup))
    add_urls = extract_url(soup)
    desc_list.extend(extract_desc(add_urls))
    
    # counter = len(job_title_list)
    # if counter >= max_counter:
    #     break
    
indeed_df = pd.DataFrame(list(zip(job_title_list, company_list, location_list, 
                                  salary_list, desc_list)), 
                         columns = ['job_title', 'company', 'location', 'salary', 'description'])

935
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=0
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=10
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=20
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=30
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=40
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=50
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=60
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=70
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=80
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=90


In [17]:
indeed_df = indeed_df.drop_duplicates(keep=False) 
indeed_df = indeed_df.reset_index(drop=True)

In [18]:
indeed_df

,job_title,company,location,salary,description
0,Machine Learning Engineer,Triplebyte,None,"$125,000 - $190,000 a year",About TriplebyteTriplebyte helps companies fin...
1,Statistical Programmer,Quantum Leap Healthcare,None,None,Statistical programmer is responsible for load...
2,Data Scientist,Blue Owl,None,"$200,000 - $350,000 a year",A million people a year die in car collisions ...
3,Data Scientist,Stripe,None,None,Generate insights and impact from data.\n\nWe'...
4,Data Scientist,Grammarly,None,None,The opportunity\n\n\nGrammarly empowers people...
5,Senior Data Analyst,OneDotSix Consulting Inc,None,"$120,000 a year",10+ years of experience in Business and Data A...
6,Data Scientist,Affinity,None,None,Affinity is on a mission to help everyone in t...
7,Research Data Scientist,University of California San Francisco,"San Francisco, CA 94143 (Inner Sunset area)",None,The Research Data Scientist will join a collab...
8,Data Scientist 1,Autodesk,"San Francisco, CA",None,"Autodesk enables our customers to imagine, des..."
9,Data Scientist,University of California San Francisco,"San Francisco, CA 94143 (Inner Sunset area)",None,The Bioinformatics Programmer will work with t...


In [87]:
str = indeed_df['salary'][0]
sal_split = re.findall(r'\d+', str.replace(",", ""))
print(sal_split)

['125000', '190000']


In [88]:
len(sal_split)

2

In [89]:
if len(sal_split) == 2:
    low_salary = int(sal_split[0])
    high_salary = int(sal_split[1])
else:
    low_salary = None
    high_salary = None

print(low_salary, high_salary)

125000 190000


In [97]:
def convert_salary(sal_str, s_flag):
    
    sal_split = re.findall(r'\d+', sal_str.replace(",", ""))
    
    if len(sal_split) == 2:
        low_salary = int(sal_split[0])
        high_salary = int(sal_split[1])
    else:
        low_salary = None
        high_salary = None
    
    if s_flag == 'l':
        salary = low_salary
    else:
        salary = high_salary
    
    return salary

indeed_df['low_salary'] = indeed_df.apply(lambda x: convert_salary(x['salary'], 'l'), axis=1)
indeed_df['high_salary'] = indeed_df.apply(lambda x: convert_salary(x['salary'], 'h'), axis=1)

indeed_df.head()
                                           

,job_title,company,location,salary,description,low_salary,high_salary
0,Machine Learning Engineer,Triplebyte,None,"$125,000 - $190,000 a year",About TriplebyteTriplebyte helps companies fin...,125000.0,190000.0
1,Statistical Programmer,Quantum Leap Healthcare,None,None,Statistical programmer is responsible for load...,NaN,NaN
2,Data Scientist,Blue Owl,None,"$200,000 - $350,000 a year",A million people a year die in car collisions ...,200000.0,350000.0
3,Data Scientist,Stripe,None,None,Generate insights and impact from data.\n\nWe'...,NaN,NaN
4,Data Scientist,Grammarly,None,None,The opportunity\n\n\nGrammarly empowers people...,NaN,NaN


In [98]:
indeed_df

,job_title,company,location,salary,description,low_salary,high_salary
0,Machine Learning Engineer,Triplebyte,None,"$125,000 - $190,000 a year",About TriplebyteTriplebyte helps companies fin...,125000.0,190000.0
1,Statistical Programmer,Quantum Leap Healthcare,None,None,Statistical programmer is responsible for load...,NaN,NaN
2,Data Scientist,Blue Owl,None,"$200,000 - $350,000 a year",A million people a year die in car collisions ...,200000.0,350000.0
3,Data Scientist,Stripe,None,None,Generate insights and impact from data.\n\nWe'...,NaN,NaN
4,Data Scientist,Grammarly,None,None,The opportunity\n\n\nGrammarly empowers people...,NaN,NaN
5,Senior Data Analyst,OneDotSix Consulting Inc,None,"$120,000 a year",10+ years of experience in Business and Data A...,NaN,NaN
6,Data Scientist,Affinity,None,None,Affinity is on a mission to help everyone in t...,NaN,NaN
7,Research Data Scientist,University of California San Francisco,"San Francisco, CA 94143 (Inner Sunset area)",None,The Research Data Scientist will join a collab...,NaN,NaN
8,Data Scientist 1,Autodesk,"San Francisco, CA",None,"Autodesk enables our customers to imagine, des...",NaN,NaN
9,Data Scientist,University of California San Francisco,"San Francisco, CA 94143 (Inner Sunset area)",None,The Bioinformatics Programmer will work with t...,NaN,NaN
